In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

ejList = pd.read_xml(
    path_or_buffer="../../../data_test/finess/simple/finess_cs1400101_stock_20230308-0330.xml",
    xpath="./structureej",
)[["nofiness"]]

eTList = pd.read_xml(
    path_or_buffer="../../../data_test/finess/simple/finess_cs1400102_stock_20230308-0331.xml",
    xpath="./structureet",
)[["nofinesset"]]

men_pmsi = pd.read_csv("../../../data_test/diamant/MEN_PMSI_ANNUEL_2022_10_25.CSV", sep=";")

men_pmsi.head()


In [ ]:
# Nombre de ligne séjours HAD à partir de 2018
had = men_pmsi[["Finess EJ", "Finess", "Année", "Nombre total de séjours HAD"]]
had = had.loc[had["Année"] >= 2018]
len(had)

In [ ]:
had["Nombre total de séjours HAD"].info()

In [ ]:
# Répartition des valeurs HAD
had.boxplot(column=["Nombre total de séjours HAD"], by="Année").set_yscale("log")

had.sort_values(by=["Nombre total de séjours HAD"], ascending=False).head(40)

# ==> Quelques établissements font beaucoup de HAD ( vérifier la cohérence ?)
# ex 920813623,750712184,690000880,750042459,330000340,690788930,690788930


In [ ]:
# Répartition des données par Année
not_null_values = had[had["Nombre total de séjours HAD"].notnull()]

had_par_annee = not_null_values.groupby(["Année"])[["Nombre total de séjours HAD"]].count()
had_par_ej_annee = not_null_values.groupby(["Année", "Finess EJ"])[["Nombre total de séjours HAD"]].count()
had_par_ej = not_null_values.groupby(["Finess EJ"])[["Nombre total de séjours HAD"]].count()
display(had_par_annee)
display(had_par_ej_annee)
display(had_par_ej)

# ==> Pas de données pour 2022
# ==> Quasiment tous les EJ n'ont qu'un seul ET avec HAD ( sauf Réunion et Guyane ~ 4 EJ)
# ==> Seulement 288 EJ avec au moins une donnée HAD

In [ ]:
# Pourcentage des valeurs N/A dans la colonne Nombre total de sejours HAD (chaque annee, EJ, ET)

na_had = had["Nombre total de séjours HAD"].isna().sum()

total_had = len(had)

pourcentage_na = na_had / total_had * 100

print("Percentage of Na in dataset is: ", pourcentage_na)

In [ ]:
# Pourcentage des valeurs N/A HAD sur chaque EJ PAR ANNEE
had_grouped_annee = had.groupby(["Finess EJ", "Année"])["Nombre total de séjours HAD"].sum()
number_of_nan = (had_grouped_annee == 0).sum()
total = len(had_grouped_annee)
(number_of_nan / total) * 100
# ==> 88,4% des Année par EJ n'ont aucune donnée

In [ ]:
# Pourcentage des valeurs N/A HAD sur chaque EJ
had_grouped_ej = had.groupby(["Finess EJ"])["Nombre total de séjours HAD"].sum()
number_of_nan = (had_grouped_ej == 0).sum()
total = len(had_grouped_ej)
(number_of_nan / total) * 100

# ==> 88,3% des EJ n'ont aucune donnée HAD

In [ ]:
# Total nombre de 0
zero_had = (had["Nombre total de séjours HAD"] == 0).sum()
print("Total number rows with value 0 is: ", zero_had)
# => Pas de valeur à 0 que des NaN

In [ ]:
# Total de valeur négative
negative_had = (had["Nombre total de séjours HAD"] < 0).sum()

print("Total number rows with value negative is: ", negative_had)
# => Pas de valeur négative

In [ ]:
# Y'a t'il de doublons :
had.duplicated().sum()
# => Non

In [ ]:
# Vérifier que les Finess EJ ont bien 9 chiffres
# Transforme le Finess en string, compte le nombre de charactère de chaque finess, ne garde que les valeurs unique du compte
had["Finess EJ"].apply(str).str.len().unique()

# numero finess est un type Objet
# numero finess a 9 chiffres --> need to handle 0 at beginning at import


In [ ]:
# Numero Finess qui existe dans ET

ET = had["Finess"]

EJ = had["Finess EJ"]

display(EJ.drop_duplicates().isin(ET).sum())
display(ET.drop_duplicates().isin(EJ).sum())
display()

# => 194 EJ qui sont dans Finess ET

In [ ]:
had[EJ.isin(ET)]
# EJs that exist also in ET column eg: 020000295, 030780100, 030780100, 940016819, 940110042
# Note that there are some years where the EJ and ET are the same value


In [ ]:
had[had["Finess"].isna()]
# number of rows where ET finess number is empty = 72
# Dans tous ces cas on a pas de donnée HAD

In [ ]:
had["Finess EJ"].isna().sum()
# All Finess EJ rows have a value

In [ ]:
# Est-ce que tous les Finess EJ correspondent bien à des vrais EJ
ejHAD = had["Finess EJ"].apply(str).drop_duplicates()
ejListStr = ejList["nofiness"].apply(str)
ejHAD.isin(ejListStr).sum()
# => Tous les EJ dans MEN_PMSI existe


In [ ]:
# Vérifier que les Et existent
et = had["Finess"].drop_duplicates().dropna()
existingEt = et.isin(eTList["nofinesset"]).sum()
totalEt = len(et)

display("Nombre d'ET qui ne sont pas des ET : " + str(totalEt - existingEt))

display(et[~et.isin(eTList["nofinesset"])].drop_duplicates())